# Продажа автомобиля

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости.

## 1. Загрузка и подготовка данных.

In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier


autos = pd.read_csv('/datasets/autos.csv')

print(autos.info())
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB
None


,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Признаки
- DateCrawled — дата скачивания анкеты из базы
- VehicleType — тип автомобильного кузова
- RegistrationYear — год регистрации автомобиля
- Gearbox — тип коробки передач
- Power — мощность (л. с.)
- Model — модель автомобиля
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- FuelType — тип топлива
- Brand — марка автомобиля
- NotRepaired — была машина в ремонте или нет
- DateCreated — дата создания анкеты
- NumberOfPictures — количество фотографий автомобиля
- PostalCode — почтовый индекс владельца анкеты (пользователя)
- LastSeen — дата последней активности пользователя

Целевой признак
- Price — цена (евро)

Сначала разберемся с аномальными значениями в целевом признаке.

In [166]:
autos['Price'].value_counts().sort_index(ascending=False)

20000      268
19999      278
19998        6
19997        1
19995       10
         ...  
4            1
3            8
2           12
1         1189
0        10772
Name: Price, Length: 3731, dtype: int64

Значение 0 в целевом признаке очень подозрителен, как и 1, и 2... 

На сайте Ebay Cars наименьшая цена, по которой продавалась машина - 150 фунтов. Возьмем это число как нижняя граница. Верхняя в 20000 кажется реальной.

In [167]:
good_autos = autos.query('Price > 150')
good_autos.shape

(337830, 16)

Исследуем столбец Power на аномальные значения.

In [168]:
autos['Power'].value_counts().sort_index(ascending=False)

20000        1
19312        1
19211        1
19208        1
17932        1
         ...  
4           30
3            9
2           10
1           34
0        40225
Name: Power, Length: 712, dtype: int64

На данный момент максимально возможное значение лошадиных сил в машине приблизительно 1500, а наименьшее - 66. Воспользуемся этими значениями для фильтрации.

In [169]:
good_autos = good_autos.query('Power >= 66 and Power < 1500')
good_autos.shape

(261766, 16)

Теперь посмотрим на пропуски.

In [170]:
#Подсчет количества пропусков в каждом столбцу
null_counts = good_autos.isnull().sum()
#Пропуски в каждом столбце в процентах
null_pct = null_counts / good_autos.shape[0] * 100

#Объединим все в один датафрейм
null_df = pd.DataFrame({'Количество пропусков':null_counts,'В процентах':null_pct})
null_df = null_df.T.astype(int)
null_df

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
Количество пропусков,0,0,16785,0,4165,0,9922,0,0,15305,0,37315,0,0,0,0
В процентах,0,0,6,0,1,0,3,0,0,5,0,14,0,0,0,0


Пропуски содержатся в следующих столбцах: VehicleType, Gearbox, Model, FuelType, NotRepaired. Все они содержат категориальный тип переменных, поэтому пропуски можно либо заполнить самым частым значением, либо напросто отбросить строки с пропусками в данных столбцах. Для принятия решения подробнее рассмотрим значения во всех этих признаках.

In [171]:
print('Количество уникальных значений в VehicleType: ' + str(good_autos['VehicleType'].unique().shape[0]))
print('Количество уникальных значений в GearBox: ' + str(good_autos['Gearbox'].unique().shape[0]))
print('Количество уникальных значений в FuelType: ' + str(good_autos['FuelType'].unique().shape[0]))
print('Количество уникальных значений в NotRepaired: ' + str(good_autos['NotRepaired'].unique().shape[0]))
print()

print(good_autos['Gearbox'].value_counts())
print(good_autos['NotRepaired'].value_counts())

Количество уникальных значений в VehicleType: 9
Количество уникальных значений в GearBox: 3
Количество уникальных значений в FuelType: 8
Количество уникальных значений в NotRepaired: 3

manual    200569
auto       57032
Name: Gearbox, dtype: int64
no     200580
yes     23871
Name: NotRepaired, dtype: int64


Пропуски в столбцах GearBox и NotRepaired заполним самым частым значением. Ручная коробка передач встречается в приблизительно 76% объявлений и значение "no" в столбце NotRepaired примерно также.

Пропуски в других столбцах просто удалим.

In [172]:
good_autos['Gearbox'].fillna('manual',inplace=True)
good_autos['NotRepaired'].fillna('no',inplace=True)
good_autos = good_autos.dropna().reset_index(drop=True)
good_autos.shape[0]

230300

In [173]:
good_autos['NumberOfPictures'].value_counts()

0    230300
Name: NumberOfPictures, dtype: int64

Избавимся также от столбца NumberOfPictures так как в нем только нули.

In [174]:
good_autos.drop('NumberOfPictures', axis=1,inplace=True)
good_autos.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,PostalCode,LastSeen
0,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,no,2016-03-14 00:00:00,90480,2016-04-05 12:47:46
1,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,91074,2016-03-17 17:40:17
2,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,60437,2016-04-06 10:17:21
3,2016-04-04 17:36:23,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,2016-04-04 00:00:00,33775,2016-04-06 19:17:07
4,2016-04-01 20:48:51,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,2016-04-01 00:00:00,67112,2016-04-05 18:18:39


Взглянем что у нас с датами

In [175]:
print('Date Crawled')
print(good_autos['DateCrawled'].str[:10].value_counts(normalize=True,dropna=False).sort_index(ascending=False).head())
print(good_autos['DateCrawled'].str[:10].value_counts(normalize=True,dropna=False).sort_index(ascending=False).tail())
print()
print('Last Seen')
print(good_autos['LastSeen'].str[:10].value_counts(normalize=True,dropna=False).sort_index(ascending=False).head())
print(good_autos['LastSeen'].str[:10].value_counts(normalize=True,dropna=False).sort_index(ascending=False).tail())
print()
print('Date created')
print(good_autos['DateCreated'].str[:10].value_counts(normalize=True,dropna=False).sort_index(ascending=False).head())
print(good_autos['DateCreated'].str[:10].value_counts(normalize=True,dropna=False).sort_index(ascending=False).tail())

good_autos['RegistrationYear'].value_counts().sort_index(ascending=False)

Date Crawled
2016-04-07    0.001611
2016-04-06    0.003222
2016-04-05    0.012393
2016-04-04    0.037920
2016-04-03    0.039640
Name: DateCrawled, dtype: float64
2016-03-09    0.033556
2016-03-08    0.033178
2016-03-07    0.035849
2016-03-06    0.014590
2016-03-05    0.026418
Name: DateCrawled, dtype: float64

Last Seen
2016-04-07    0.134429
2016-04-06    0.226561
2016-04-05    0.130651
2016-04-04    0.025041
2016-04-03    0.025106
Name: LastSeen, dtype: float64
2016-03-09    0.009184
2016-03-08    0.007438
2016-03-07    0.004902
2016-03-06    0.003943
2016-03-05    0.001203
Name: LastSeen, dtype: float64

Date created
2016-04-07    0.001546
2016-04-06    0.003200
2016-04-05    0.011246
2016-04-04    0.037907
2016-04-03    0.039887
Name: DateCreated, dtype: float64
2015-11-02    0.000004
2015-09-09    0.000004
2015-08-10    0.000004
2015-08-07    0.000004
2015-03-20    0.000004
Name: DateCreated, dtype: float64


2018       2
2017       7
2016     128
2015     949
2014    2215
        ... 
1949       1
1947       1
1942       2
1937       1
1910       1
Name: RegistrationYear, Length: 73, dtype: int64

Диапазон дат создания объявления 2015-2016 год, а в датасете присутствуют машины, зарегистрированные в 2017 и 2018 году. Избивляемся от них.

In [176]:
good_autos = good_autos[good_autos['RegistrationYear'].between(1909,2016)]

Что там с месяцем регистрации

In [177]:
good_autos['RegistrationMonth'].unique()

array([ 8,  6,  7, 10, 12, 11,  3,  2,  1,  4,  9,  5,  0], dtype=int64)

Месяц должен варьироваться либо между 0 и 11, ибо между 1 и 12. У нас значение лежит между 0 и 12, значит либо 0 либо 12 некорректное значение. Так как формы заполняются обычными людьми, то вряд ли они будум именовать первый месяц через 0, следовательно, удалим все строки со значением регистрационного месца равного нулю.

In [178]:
good_autos = good_autos[good_autos['RegistrationMonth'] != 0]
good_autos['RegistrationMonth'].unique()

array([ 8,  6,  7, 10, 12, 11,  3,  2,  1,  4,  9,  5], dtype=int64)

Объединим год регистрации с месяцем

In [179]:
good_autos['RegistrationMonth'] = good_autos['RegistrationMonth'].astype(str)
good_autos['RegistrationMonth'] = good_autos['RegistrationMonth'].apply(lambda x: x.zfill(2))
good_autos['RegistrationYear'] = good_autos['RegistrationYear'].astype(str)
good_autos['RegistrationDate'] = good_autos['RegistrationYear'] + '-' + good_autos['RegistrationMonth']
good_autos.drop(['RegistrationMonth','RegistrationYear'], axis=1,inplace=True)

good_autos.head()

,DateCrawled,Price,VehicleType,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired,DateCreated,PostalCode,LastSeen,RegistrationDate
0,2016-03-14 12:52:21,9800,suv,auto,163,grand,125000,gasoline,jeep,no,2016-03-14 00:00:00,90480,2016-04-05 12:47:46,2004-08
1,2016-03-17 16:54:04,1500,small,manual,75,golf,150000,petrol,volkswagen,no,2016-03-17 00:00:00,91074,2016-03-17 17:40:17,2001-06
2,2016-03-31 17:25:20,3600,small,manual,69,fabia,90000,gasoline,skoda,no,2016-03-31 00:00:00,60437,2016-04-06 10:17:21,2008-07
3,2016-04-04 17:36:23,650,sedan,manual,102,3er,150000,petrol,bmw,yes,2016-04-04 00:00:00,33775,2016-04-06 19:17:07,1995-10
4,2016-04-01 20:48:51,2200,convertible,manual,109,2_reihe,150000,petrol,peugeot,no,2016-04-01 00:00:00,67112,2016-04-05 18:18:39,2004-08


Оптимизируем память

In [180]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        elif 'Date' in col or 'Seen' in col:
            df[col] = pd.to_datetime(df[col])
        else:
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

In [181]:
good_autos = reduce_mem_usage(good_autos)
print()
good_autos.info()

Потребление памяти меньше на 12.9 Мб (минус 50.8 %)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 222057 entries, 0 to 230299
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   DateCrawled       222057 non-null  datetime64[ns]
 1   Price             222057 non-null  int16         
 2   VehicleType       222057 non-null  category      
 3   Gearbox           222057 non-null  category      
 4   Power             222057 non-null  int16         
 5   Model             222057 non-null  category      
 6   Kilometer         222057 non-null  int32         
 7   FuelType          222057 non-null  category      
 8   Brand             222057 non-null  category      
 9   NotRepaired       222057 non-null  category      
 10  DateCreated       222057 non-null  datetime64[ns]
 11  PostalCode        222057 non-null  int32         
 12  LastSeen          222057 non-null  datetime64[ns]
 13  Regist

Посмотрим внимательно на колонку "Kilometer"

In [183]:
good_autos['Kilometer'].unique()

array([125000, 150000,  90000,  30000,  70000, 100000,  60000,  20000,
        50000,  40000,   5000,  10000,  80000])

Несмотря на то, что данный признак числовой, уникальных значений у него не много. Похоже на сайте продавцам позволяют выбрать пробег из списка предопределенных значений. Взглянем на распределение цен для каждого значения в этмо столбце

In [184]:
good_autos.pivot_table(index='Kilometer',values='Price',aggfunc='mean')

,Price
Kilometer,
5000,6367.526353
10000,12513.539510
20000,10982.265347
30000,10755.453135
40000,10794.547956
50000,10403.236028
60000,9903.910621
70000,9264.753218
80000,8747.562724


Начиная с 10000 цена снижается, хотя и незначительно между 10000 и 50000.

In [185]:
num_features = ['Price','Power','Kilometer','PostalCode']
corr_df = good_autos[num_features]
corr_df.corr(method='pearson')

,Price,Power,Kilometer,PostalCode
Price,1.000000,0.415207,-0.431868,0.053461
Power,0.415207,1.000000,0.115667,0.031949
Kilometer,-0.431868,0.115667,1.000000,-0.013837
PostalCode,0.053461,0.031949,-0.013837,1.000000


Достаточно сильная положительная корреляция между ценой и количеством л.с., что вполне логично. И отрицательная с километражем. Столбец PostalCode нам вряд ли пригодится.

Теперь выберем признаки для последующих моделей и произведем OHE-Encoding категориальных переменных.

In [186]:
lr_columns = ['VehicleType','Gearbox','Power','Model','Kilometer','FuelType','Brand','NotRepaired','Price']
good_autos_enc = pd.get_dummies(good_autos[lr_columns], drop_first=True)
good_autos_enc

,Power,Kilometer,Price,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,VehicleType_suv,VehicleType_wagon,...,Brand_saab,Brand_seat,Brand_skoda,Brand_smart,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_volkswagen,Brand_volvo,NotRepaired_yes
0,163,125000,9800,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,75,150000,1500,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,69,90000,3600,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
3,102,150000,650,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,109,150000,2200,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230295,140,150000,7900,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
230296,225,150000,3200,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1
230297,101,125000,1199,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
230298,102,150000,9200,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


## 2. Обучим разные модели с различными гиперпараметрами

Разбиваем данные

In [187]:
features = good_autos_enc.drop('Price',axis=1)
target   = good_autos_enc['Price']
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=42)

Опробуем Линейную Регрессию

In [188]:
model = LinearRegression()
model.fit(features_train,target_train)

mse = cross_val_score(model,features_test,target_test,scoring='neg_mean_squared_error',cv=10)
mse = np.mean(mse)
np.sqrt(abs(mse))

3069.5046444666286

Данная модель дала нам RMSE = 3069. Будем пытаться уменьшить ее.

Для моделей LightGBM и CatBoost воспользуемся GridSearchCV и RandomizedSearchCV, чтобы подобрать оптимальные значения гиперпараметров. Напишем функцию, которая будет использовать один из этих методов для подбора гиперпараметров, а также обучать модель и делать предсказания. Также она будет засекать время своего выполнения.

In [189]:
def search_learn_predict(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv=10, scoring_fit='neg_mean_squared_error',
                         search_mode = 'GridSearchCV', n_iterations = 0):
    start = time.time()
    
    fitted_model = None
    
    if(search_mode == 'GridSearchCV'):
        gs = GridSearchCV(
            estimator=model,
            param_grid=param_grid, 
            cv=cv, 
            n_jobs=-1, 
            scoring=scoring_fit,
            verbose=2
        )
        fitted_model = gs.fit(X_train_data, y_train_data)
    else:
        rs = RandomizedSearchCV(
            estimator=model,
            param_distributions=param_grid, 
            cv=cv,
            n_iter=n_iterations,
            n_jobs=-1, 
            scoring=scoring_fit,
            verbose=2
        )
        fitted_model = rs.fit(X_train_data, y_train_data)
        
    pred = fitted_model.predict(X_test_data)
    end = time.time()
    print('Time of execution: {}'.format(end - start))
    return fitted_model, pred

## LGBM

Используем GridSearchCV

In [190]:
model = lgb.LGBMRegressor()
param_grid = {
    'n_estimators': [20, 50, 100],
    'max_depth': [4, 6, 8],
    'num_leaves': [50, 100]
}

model, pred = search_learn_predict(features_train, features_test, target_train, target_test, model, 
                                 param_grid, cv=5)

print(np.sqrt(abs(model.best_score_)))
print(model.best_params_)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   42.5s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  1.9min finished


Время выполнения функции: 116.10732126235962
2450.472699016674
{'max_depth': 8, 'n_estimators': 100, 'num_leaves': 100}


Используем RandomizedSearchCV

In [191]:
model = lgb.LGBMRegressor()
param_grid = {
    'n_estimators': [20, 50, 100],
    'max_depth': [4, 6, 8],
    'num_leaves': [50, 100]
}

model, pred = search_learn_predict(features_train, features_test, target_train, target_test, model, 
                                 param_grid, cv=5, search_mode = 'RandomizedSearchCV', n_iterations = 10 )

print(np.sqrt(abs(model.best_score_)))
print(model.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   45.0s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.0min finished


Время выполнения функции: 65.30004692077637
2450.472699016674
{'num_leaves': 100, 'n_estimators': 100, 'max_depth': 8}


## CatBoost

GridSearchCV

In [194]:
model = CatBoostRegressor()
param_grid = {
    'depth': [6,8,10],
    'learning_rate': [0.01,0.05,0.1],
    'iterations': [10,30,50]
}
model, pred = search_learn_predict(features_train, features_test, target_train, target_test, model, 
                                 param_grid, cv=5)

print(np.sqrt(abs(model.best_score_)))
print(model.best_params_)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   59.7s
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:  4.6min finished


0:	learn: 4481.7814295	total: 57.6ms	remaining: 2.82s
1:	learn: 4245.2817983	total: 114ms	remaining: 2.75s
2:	learn: 4044.5484996	total: 169ms	remaining: 2.64s
3:	learn: 3869.8471186	total: 225ms	remaining: 2.59s
4:	learn: 3718.7932239	total: 291ms	remaining: 2.62s
5:	learn: 3590.4354623	total: 348ms	remaining: 2.55s
6:	learn: 3483.9808293	total: 409ms	remaining: 2.51s
7:	learn: 3391.8261057	total: 468ms	remaining: 2.46s
8:	learn: 3308.1861392	total: 532ms	remaining: 2.42s
9:	learn: 3237.2968218	total: 591ms	remaining: 2.36s
10:	learn: 3176.7090424	total: 647ms	remaining: 2.29s
11:	learn: 3122.2612618	total: 705ms	remaining: 2.23s
12:	learn: 3077.3001121	total: 770ms	remaining: 2.19s
13:	learn: 3038.8133698	total: 826ms	remaining: 2.12s
14:	learn: 3006.6804064	total: 886ms	remaining: 2.07s
15:	learn: 2978.6117823	total: 960ms	remaining: 2.04s
16:	learn: 2951.9175923	total: 1.01s	remaining: 1.97s
17:	learn: 2929.1411092	total: 1.07s	remaining: 1.91s
18:	learn: 2908.2538946	total: 1.13s	

RandomizedSearchCV

In [195]:
model = CatBoostRegressor()
param_grid = {
    'depth': [6,8,10],
    'learning_rate': [0.01,0.05,0.1],
    'iterations': [10,30,50]
}
model, pred = search_learn_predict(features_train, features_test, target_train, target_test, model, 
                                 param_grid, cv=5, search_mode = 'RandomizedSearchCV', n_iterations = 10 )

print(np.sqrt(abs(model.best_score_)))
print(model.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.6min finished


0:	learn: 4487.9277678	total: 40ms	remaining: 1.96s
1:	learn: 4257.7071061	total: 73.8ms	remaining: 1.77s
2:	learn: 4062.2074436	total: 107ms	remaining: 1.68s
3:	learn: 3893.9829021	total: 141ms	remaining: 1.62s
4:	learn: 3750.0242633	total: 173ms	remaining: 1.56s
5:	learn: 3627.8756499	total: 205ms	remaining: 1.51s
6:	learn: 3527.5530301	total: 252ms	remaining: 1.54s
7:	learn: 3440.0991882	total: 295ms	remaining: 1.55s
8:	learn: 3360.8901686	total: 331ms	remaining: 1.51s
9:	learn: 3295.0210766	total: 365ms	remaining: 1.46s
10:	learn: 3237.1162348	total: 399ms	remaining: 1.41s
11:	learn: 3185.0579429	total: 444ms	remaining: 1.4s
12:	learn: 3141.4402652	total: 482ms	remaining: 1.37s
13:	learn: 3105.5200246	total: 515ms	remaining: 1.32s
14:	learn: 3073.1671468	total: 547ms	remaining: 1.27s
15:	learn: 3042.1029552	total: 579ms	remaining: 1.23s
16:	learn: 3014.4994946	total: 613ms	remaining: 1.19s
17:	learn: 2992.8648450	total: 645ms	remaining: 1.15s
18:	learn: 2972.0806130	total: 683ms	re

## 3. Проанализируем скорость работы и качество моделей.

Как LightGBM, так и CatBoost дают значение метрики лучшее, чем Линейная Регрессия. У последней RMSE вышло 3069. CatBoost дал значение 2691 при использовании GridSearchCV (281 секунда) и 2749 при использовании RandomizedSearchCV (101 секунда). Лучшее значение у LightGBM - 2450, независимо от метода подбора. Помимо лучшего значения метрики данная модель выигрывает в скорости (65 секунд при RandomizedSearchCV и 116 при GridSearchCV). 